In [66]:
from hydradx.model.amm.omnipool_amm import OmnipoolState
from hydradx.model.amm.agents import Agent




In [67]:
dot_usdt_order_book = {
    'bids': [{'price': 3.80,'amount': 200},
             {'price': 3.79,'amount': 100},
             {'price': 3.70,'amount': 100}],
    'asks': [{'price': 3.90,'amount': 100},
             {'price': 4.00,'amount': 200}]
}
dot_usdt_order_book = {
    'bids': [{'price': 3.60, 'amount': 200},
             {'price': 3.59, 'amount': 100},
             {'price': 3.50, 'amount': 100},
             {'price': 3.40, 'amount': 2000}],
    'asks': [{'price': 3.70, 'amount': 100},
             {'price': 3.74, 'amount': 5000},
             {'price': 3.80, 'amount': 200},
             {'price': 3.90, 'amount': 2000}]
}

hdx_usdt_order_book = {
    'bids': [{'price': 0.03, 'amount': 2000},
             {'price': 0.025, 'amount': 2000},
             {'price': 0.02, 'amount': 2000},
             {'price': 0.015, 'amount': 2000}],
    'asks': [{'price': 0.04, 'amount': 2000},
             {'price': 0.05, 'amount': 2000},
             {'price': 0.06, 'amount': 2000},
             {'price': 0.07, 'amount': 2000}]
}

hdx_dot_order_book = {
    'bids': [{'price': 0.005, 'amount': 2000},
             {'price': 0.004, 'amount': 2000}],
    'asks': [{'price': 0.0052, 'amount': 2000},
             {'price': 0.0055, 'amount': 2000}]
}

order_book = {
    ('DOT','USDT'): dot_usdt_order_book,
    ('HDX','USDT'): hdx_usdt_order_book,
    ('HDX','DOT'): hdx_dot_order_book
}

tokens = {
    'USDT': {
        'liquidity': 1000000,
        'LRNA': 100000
    },
    'DOT': {
        'liquidity': 800000,
        'LRNA': 300000
    },
    'HDX': {
        'liquidity': 200000000,
        'LRNA': 400000
    }
}

op_state = OmnipoolState(
            tokens=tokens,
            lrna_fee=0.00,
            asset_fee=0.00,
    preferred_stablecoin='USDT',
        )

init_state = op_state.copy()

OmnipoolState.price(op_state, 'DOT', 'USDT')
OmnipoolState.price(op_state, 'HDX', 'USDT')
OmnipoolState.price(op_state, 'HDX', 'DOT')

0.005333333333333333

In [68]:
all_swaps = {}
for tkn_pair in order_book:
    pair_order_book = order_book[tkn_pair]
    tkn = tkn_pair[0]
    numeraire = tkn_pair[1]
    bids = pair_order_book['bids']
    asks = pair_order_book['asks']
    bids = sorted(bids, key=lambda x: x['price'], reverse=True)
    asks = sorted(asks, key=lambda x: x['price'], reverse=False)
    
    agent = Agent(holdings={'USDT': 1000000, 'DOT': 1000000, 'HDX': 1000000}, unique_id='bot')
    
    op_spot = OmnipoolState.price(op_state, tkn, numeraire)
    i = 0
    profit = 0
    bid_executed = False
    ask_executed = False
    last_profit = 0
    if op_spot < bids[i]['price']:
        bid_executed = True
        test_state = op_state
        test_agent = agent
        while i < len(bids) and op_spot < bids[i]['price']:  # spot is lower than the highest bid, so we should buy from Omnipool and sell to CEX
            print('buy')
            op_state = test_state
            agent = test_agent
            # profit += last_profit
            # init_usdt = agent.holdings[numeraire]
            # init_dot = agent.holdings[tkn]
            test_state = op_state.copy()
            test_agent = agent.copy()
            test_state.swap(test_agent, tkn_buy=tkn, tkn_sell=numeraire, buy_quantity=bids[i]['amount'])
            op_spot = OmnipoolState.price(test_state, tkn, numeraire)
            # usdt_in = init_usdt - test_agent.holdings[tkn]
            # last_profit = bids[i]['price'] * bids[i]['amount'] - usdt_in
            i += 1
    elif op_spot > asks[i]['price']:
        ask_executed = True
        test_state = op_state
        test_agent = agent
        while i < len(bids) and op_spot > asks[i]['price']:  # spot is higher than the lowest ask, so we should buy from CEX and sell to Omnipool
            print('sell')
            op_state = test_state
            agent = test_agent
            # profit += last_profit
            # init_usdt = agent.holdings[numeraire]
            # init_dot = agent.holdings[tkn]
            test_state = op_state.copy()
            test_agent = agent.copy()
            test_state.swap(test_agent, tkn_buy=numeraire, tkn_sell=tkn, sell_quantity=asks[i]['amount'])
            op_spot = OmnipoolState.price(test_state, tkn, numeraire)
            # usdt_out = test_agent.holdings[numeraire] - init_usdt 
            # last_profit = usdt_out - asks[i]['price'] * asks[i]['amount'] 
            i += 1
    
    swaps = []
    if bid_executed:
        if op_spot < bids[i-1]['price']:  # last trade can be fully executed without moving spot price too much
            swaps = [('buy',bids[j]) for j in range(i)]
            op_state.swap(agent, tkn_buy=tkn, tkn_sell=numeraire, buy_quantity=bids[i-1]['amount'])
            # profit += last_profit
        else:  # Use binary search to partially fill last order
            swaps = bids[0:i-1]
            swaps = [('buy',bids[j]) for j in range(i-1)]
            bid = bids[i-1]
            amt_high = bid['amount']
            amt_low = 0
            iters = 100
            # init_usdt = agent.holdings[numeraire]
            # init_dot = agent.holdings[tkn]
            for i in range(iters):
                test_agent = agent.copy()
                test_state = op_state.copy()
                amt = amt_low + (amt_high - amt_low) / 2
                test_state.swap(test_agent, tkn_buy=tkn, tkn_sell=numeraire, buy_quantity=amt)
                op_spot = OmnipoolState.price(test_state, tkn, numeraire)
                # usdt_in = init_usdt - test_agent.holdings[numeraire]
                # profit = bid['price'] * amt - usdt_in
                if op_spot > bid['price']:
                    amt_high = amt
                elif op_spot < bid['price']:
                    amt_low = amt
                else:
                    break
                    
            op_state.swap(agent, tkn_buy='DOT', tkn_sell=numeraire, buy_quantity=amt_low)
            op_spot = OmnipoolState.price(op_state, tkn, numeraire)
            swaps.append(('buy',{'price': bid['price'], 'amount': amt_low}))
            # usdt_in = init_usdt - agent.holdings[numeraire]
            # profit += bid['price'] * amt_low - usdt_in
    
    elif ask_executed:
        if op_spot > asks[i-1]['price']:  # last trade can be fully executed without moving spot price too much
            swaps = [('sell',asks[j]) for j in range(i)]
            # profit += last_profit
        else:  # Use binary search to partially fill last order
            # swaps = asks[0:i-1]
            swaps = [('sell',asks[j]) for j in range(i-1)]
            ask = asks[i-1]
            amt_high = ask['amount']
            amt_low = 0
            iters = 100
            # init_usdt = agent.holdings[numeraire]
            # init_dot = agent.holdings[tkn]
            for i in range(iters):
                test_agent = agent.copy()
                test_state = op_state.copy()
                amt = amt_low + (amt_high - amt_low) / 2
                test_state.swap(test_agent, tkn_buy=numeraire, tkn_sell=tkn, sell_quantity=amt)
                op_spot = OmnipoolState.price(test_state, tkn, numeraire)
                # usdt_out = test_agent.holdings[numeraire] - init_usdt
                # profit = usdt_out - ask['price'] * amt
                if op_spot < ask['price']:
                    amt_high = amt
                elif op_spot > ask['price']:
                    amt_low = amt
                else:
                    break
                    
            op_state.swap(agent, tkn_buy=numeraire, tkn_sell=tkn, buy_quantity=amt_low)
            op_spot = OmnipoolState.price(op_state, tkn, numeraire)
            swaps.append(('sell',{'price': ask['price'], 'amount': amt_low}))
            # usdt_out = agent.holdings[numeraire] - init_usdt
            # profit += usdt_out - ask['price'] * amt_low
    # print(profit)
    print(op_spot)
    # print(swaps)
    all_swaps[tkn_pair] = swaps
print(all_swaps)
    
    # def swap(
    #         self,
    #         agent: Agent,
    #         tkn_buy: str, tkn_sell: str,
    #         buy_quantity: float = 0,
    #         sell_quantity: float = 0,
    #         modify_imbalance: bool = True,  # this is a hack to avoid modifying the imbalance for arbitrager LRNA swaps,
    #         # since those would not actually be executed as LRNA swaps
    #         # note that we still apply the imbalance modification due to LRNA fee
    #         # collection, we just don't apply the imbalance modification from
    #         # the sale of LRNA back to the pool.
    # ):

sell
sell
3.74458182070966
buy
buy
buy
0.019984317570700173
sell
0.00533533316907185
{('DOT', 'USDT'): [('sell', {'price': 3.7, 'amount': 100}), ('sell', {'price': 3.74, 'amount': 167.2011879572528})], ('HDX', 'USDT'): [('buy', {'price': 0.03, 'amount': 2000}), ('buy', {'price': 0.025, 'amount': 2000}), ('buy', {'price': 0.02, 'amount': 2000})], ('HDX', 'DOT'): [('sell', {'price': 0.0052, 'amount': 2000})]}


In [69]:
state = init_state.copy()
for tkn_pair in all_swaps:
    swaps = all_swaps[tkn_pair]
    for swap in swaps:
        if swap[0] == 'buy':
            state.swap(agent, tkn_buy=tkn_pair[0], tkn_sell=tkn_pair[1], buy_quantity=swap[1]['amount'])
        elif swap[0] == 'sell':
            state.swap(agent, tkn_buy=tkn_pair[1], tkn_sell=tkn_pair[0], sell_quantity=swap[1]['amount'])

for tkn_pair in all_swaps:
    print(tkn_pair)
    print(OmnipoolState.price(state, tkn_pair[0], tkn_pair[1]))

('DOT', 'USDT')
3.7409966862076747
('HDX', 'USDT')
0.019965578548695476
('HDX', 'DOT')
0.0053369677183368455
